In [ ]:
!pip install sentence-transformers

In [ ]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import transformers
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer("ai-forever/ru-en-RoSBERTa")

In [ ]:
raw_data = pd.read_csv('эмоции32.csv', encoding='windows-1251', sep=';')
raw_data.head(8)

In [ ]:
del raw_data['Unnamed: 10']
raw_data['Название эмоции'] = raw_data['Название эмоции'].str.lower()
raw_data.iloc[:, :-1] = raw_data.iloc[:, :-1].map(lambda x: x.replace('\n', ' '))

In [ ]:
df = pd.DataFrame()

df['Название эмоции'] = raw_data['Название эмоции']
df['Ребенок'] = raw_data['Ребенок']
df['Взрослый'] = raw_data['Взрослый']
df['Мужчина'] = raw_data['Мужчина']
df['Женщина'] = raw_data['Женщина']

df.head()

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+\.', '', text)
    text = re.sub(r'\.', '', text)
    return text

df['Ребенок'] = df['Ребенок'].apply(clean_text)
df['Взрослый'] = df['Взрослый'].apply(clean_text)
df['Мужчина'] = df['Мужчина'].apply(clean_text)
df['Женщина'] = df['Женщина'].apply(clean_text)

df.head()

In [ ]:
df[df.columns[1]][0]

In [ ]:
embed = pd.DataFrame()

embed['Название эмоции'] = df['Название эмоции']

for i in range(1, 5):
    row = []
    for j in range(32):
      temp = model.encode(df[df.columns[i]][j])
      row.append(temp)
    embed[df.columns[i]] = row

In [ ]:
len(embed.iloc[0, 1])

In [ ]:
emotions_names = list(df['Название эмоции'])
personas_names = ['Ребенок', 'Взрослый', 'Мужчина', 'Женщина']

In [ ]:
from itertools import combinations

persons = [
    {'person_id': 0, 'person_name': 'Ребенок'},
    {'person_id': 1, 'person_name': 'Взрослый'},
    {'person_id': 2, 'person_name': 'Мужчина'},
    {'person_id': 3, 'person_name': 'Женщина'}
]

emotions = emotions_names

# Создаем DataFrame всех (person, emotion)
data = []
for p in persons:
    for e in emotions:
        data.append({
            'person_id': p['person_id'],
            'person_name': p['person_name'],
            'emotion': e
        })

df = pd.DataFrame(data)

# Создаем все уникальные пары без повторений
pairs = list(combinations(df.index, 2))

# Формируем результат
result_rows = []
for i, j in pairs:
    row1 = df.loc[i]
    row2 = df.loc[j]

    idx1 = emotions.index(row1['emotion'])
    idx2 = emotions.index(row2['emotion'])

    vec1 = embed.iloc[idx1][row1['person_name']]
    vec2 = embed.iloc[idx2][row2['person_name']]

    cos_sim_value = util.cos_sim(vec1, vec2).item()

    is_diff_emotion = 1 if row1['emotion'] != row2['emotion'] else 0
    is_diff_person = 1 if row1['person_id'] != row2['person_id'] else 0

    result_rows.append({
        'person1_id': row1['person_id'],
        'person1_name': row1['person_name'],
        'person2_id': row2['person_id'],
        'person2_name': row2['person_name'],
        'emotion1': row1['emotion'],
        'emotion2': row2['emotion'],
        'cos': cos_sim_value,
        'Is_diff_emotion': is_diff_emotion,
        'Is_diff_person': is_diff_person
    })

result_df = pd.DataFrame(result_rows)

print(result_df.shape)  # (8128, 7)
result_df

In [ ]:
  def extract_gender(name):
      if name == 'Мужчина':
          return 'Male'
      elif name == 'Женщина':
          return 'Female'
      else:
          return np.nan

  def extract_age(name):
      if name == 'Ребенок':
          return 'Child'
      elif name == 'Взрослый':
          return 'Adult'
      else:
          return np.nan

  test_persons = result_df

  test_persons['person1_gender'] = test_persons['person1_name'].apply(extract_gender)
  test_persons['person2_gender'] = test_persons['person2_name'].apply(extract_gender)

  test_persons['person1_age'] = test_persons['person1_name'].apply(extract_age)
  test_persons['person2_age'] = test_persons['person2_name'].apply(extract_age)

  # Создаем признаки различий
  test_persons['Is_diff_gender'] = np.where(
      test_persons[['person1_gender', 'person2_gender']].notna().all(axis=1),
      test_persons['person1_gender'] != test_persons['person2_gender'],
      np.nan
  )

  test_persons['Is_diff_age_group'] = np.where(
      test_persons[['person1_age', 'person2_age']].notna().all(axis=1),
      test_persons['person1_age'] != test_persons['person2_age'],
      np.nan
  )

  test_persons

In [ ]:
def comparable_groups(row):
    # Возвращаем True, если обе из возрастных групп (Child, Adult) или обе из гендерных (Male, Female)
    age_groups = ['Ребенок', 'Взрослый']
    gender_groups = ['Мужчина', 'Женщина']

    p1 = row['person1_name']
    p2 = row['person2_name']

    # Если обе в возрастных — ок
    if (p1 in age_groups) and (p2 in age_groups):
        return True
    # Если обе в гендерных — ок
    if (p1 in gender_groups) and (p2 in gender_groups):
        return True

    # Иначе — разные типы, не сравниваем
    return False

result_df_filtered = test_persons[test_persons.apply(comparable_groups, axis=1)].copy()

In [19]:
result_df_filtered

,person1_id,person1_name,person2_id,person2_name,emotion1,emotion2,cos,Is_diff_emotion,Is_diff_person,person1_gender,person2_gender,person1_age,person2_age,Is_diff_gender,Is_diff_age_group
0,0,Ребенок,0,Ребенок,гнев,изумление,0.559539,1,0,NaN,NaN,Child,Child,NaN,0.0
1,0,Ребенок,0,Ребенок,гнев,отвращение,0.480251,1,0,NaN,NaN,Child,Child,NaN,0.0
2,0,Ребенок,0,Ребенок,гнев,ужас,0.712524,1,0,NaN,NaN,Child,Child,NaN,0.0
3,0,Ребенок,0,Ребенок,гнев,восхищение,0.615569,1,0,NaN,NaN,Child,Child,NaN,0.0
4,0,Ребенок,0,Ребенок,гнев,восторг,0.678856,1,0,NaN,NaN,Child,Child,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,3,Женщина,3,Женщина,разочарование,презрение,0.743977,1,0,Female,Female,NaN,NaN,0.0,NaN
8124,3,Женщина,3,Женщина,разочарование,агрессия,0.556850,1,0,Female,Female,NaN,NaN,0.0,NaN
8125,3,Женщина,3,Женщина,раскаяние,презрение,0.578382,1,0,Female,Female,NaN,NaN,0.0,NaN
8126,3,Женщина,3,Женщина,раскаяние,агрессия,0.478956,1,0,Female,Female,NaN,NaN,0.0,NaN


In [ ]:
result_df_filtered.to_csv('pairs.csv', index=False)